# Recurrent Neural Network
A simple implementation for a classification problem of predicting next character in a word
This variant uses an LSTM

## Problem description

**Goal**: Predict next characters given an imcomplete word

## Setup

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout


2025-08-17 20:02:26.731399: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-17 20:02:27.116901: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-17 20:02:28.732779: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


## Prepare train data

In [3]:
text = open(tf.keras.utils.get_file("shakespeare.txt", 
            "https://www.gutenberg.org/cache/epub/17355/pg17355.txt"), "r").read()

# setup the known alphabet
chars = sorted(set(text))
char2idx = {c: i for i,c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}

print(idx2char)

# convert text to numbers (the indexes defined in the alphabet)
encoded = np.array([char2idx[c] for c in text])

# create sequences (each sequence will have a window size)
seq_length = 100
X = []
y = []

for i in range(len(encoded) - seq_length):
    X.append(encoded[i:i+seq_length])
    y.append(encoded[i+seq_length])

X = np.array(X)
y = np.array(y)


{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


# Build the Model

In [4]:
model = Sequential([
    keras.layers.Embedding(input_dim=len(chars), output_dim=64),      # this will convert each integer in a dense vector
    keras.layers.LSTM(128, return_sequences=True),                     # RNN with higher memory capacity, can handle bigger text
    keras.layers.LSTM(128, return_sequences=False), 
    Dense(len(chars), activation='softmax')                         # softmax will output a probability for each character
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


2025-08-17 20:02:35.822837: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [ ]:
# do the training
model.fit(X, y, epochs=10, batch_size=128)


Epoch 1/10


2025-08-17 20:03:35.255350: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 892235200 exceeds 10% of free system memory.


  42/2179 ━━━━━━━━━━━━━━━━━━━━ 15:58 448ms/step - loss: 3.3289

KeyboardInterrupt: 

## Predict

In [ ]:
def predict_text(start_seq, length=500):
    seq = [char2idx[c] for c in start_seq]
    for _ in range(length):
        x = np.array([seq[-seq_length:]])   # get last seq_length chars of array
        preds = model.predict(x, verbose=0)
        next_idx = np.random.choice(len(chars), p=preds[0])  # sample instead of argmax for variety
        seq.append(next_idx)
    return ''.join(idx2char[i] for i in seq)

print(predict_text("ROMEO: "))

ROMEO: Yoke: and let's movest my foe that so:
It, Margaret this news, and never with her we-gent;
And will with thee my bid; now that, and nother
Too much moon, sir, saim my word--beike!
, ly the camed charity; I such mednom
The court, with titnes of our disper'd instrums,
Measonsime to entreat winked at he;
Here, your beauted with a tribute the more.

All Hereser:
No; sir, tough times that thou know
To pleasement and a country stowl
To be nosfipled, life a cloudy
Where no gentleman twenty miserable
Do
